# High-degree differentiation

...

We often need to perform differentiation numerically, however: perhaps because the expression isn;t available, or doesn't have a closed-form derivative, or we simply can't (or don't want to) work symbolically. In that case we need numerical differentiation.
